# Prep_300
This notebook deals with the preparation of my third research question 'What python packages are similar by description?'

## Purpose
* In this notebook I will be scraping descriptions of packages from pypi website.

    * I will begin be reading in a dataframe of unique packages
    * Then embed each package into a pypi url
    * Check the status code of each url
    * Scrape existing pages from pypi 
    * Save to separate JSON files 
    * Extract package description from each JSON file 
    * Save to dataframe

In [35]:
#importing relevant packages
import requests 
import json
import pandas as pd
import time
import csv
import sys
import numpy as np

### Reading in dataframe

In [2]:
df_import = pd.read_csv('../data/CSV_files/Imports.csv') # reading in csv 

In [3]:
df_import = pd.DataFrame(df_import['imports'])

In [4]:
df_import.to_csv('../data/CSV_files/Imports_new_RQ3.csv', index=False, encoding='utf-8')

In [5]:
with open("../data/Txt_files/imports.txt", "w") as my_output_file:
    with open('../data/CSV_files/Imports_new_RQ3.csv', "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Converting CSV file of imports to a text file

In [6]:
with open('../data/Txt_files/imports.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('../data/Txt_files/imports.txt', 'w') as fout:
    fout.writelines(data[1:])

Getting rid of first word 'imports' as it will not be needed. This was the title of a column in the CSV file.

In [7]:
f = open("../data/Txt_files/imports.txt", "r") 
print(f.read())

numpy
matplotlib.pyplot
pandas
sklearn
os
matplotlib
sklearn.metrics
math
seaborn
IPython.display
time
sys
sklearn.cross_validation
scipy
sklearn.linear_model
__future__
tensorflow
collections
random
sklearn.ensemble
sklearn.preprocessing
datetime
sklearn.model_selection
re
pickle
json
scipy.stats
pylab
csv
sklearn.tree
keras.models
itertools
keras.layers
sklearn.svm
glob
cv2
sklearn.neighbors
requests
sklearn.datasets
sklearn.feature_extraction.text
graphlab
sklearn.decomposition
sklearn.grid_search
warnings
nltk
sklearn.naive_bayes
sklearn.cluster
PIL
bs4
mpl_toolkits.mplot3d
statsmodels.api
sklearn.pipeline
ipywidgets
nltk.corpus
tqdm
string
six.moves
keras
IPython.core.display
os.path
matplotlib.image
operator
scipy.optimize
pyspark
plotly.graph_objs
matplotlib.colors
matplotlib.pylab
urllib.request
urllib
networkx
pprint
cs231n.data_utils
sklearn.feature_selection
sklearn.utils
keras.optimizers
copy
cs231n.gradient_check
utils
statsmodels.formula.api
keras.layers.core
skimage
scip

Here is the Text file of all the imports we will use later to scrape descriptions.

In [2]:
#two variables to squeeze a string between these two so it will become a full ur
part1 = 'https://pypi.org/pypi/'
part2 = '/json'

#open the outputfile before the for loop
text_file = open("../data/Txt_files/url.txt", "w")

#open the file which contains the strings
with open('../data/Txt_files/imports.txt', 'r') as f:
    for i in f:    
        uri = part1 + i.strip(' \n\t') + part2
        text_file.write(uri)
        text_file.write("\n")

text_file.close()

Embedding the Imports into a Pypi url for scraping and putting into a new text file

The cell below takes 12 hours to run

In [9]:
text_file = open("../data/Txt_files/Pypi_url.txt", "w")
with open('../data/Txt_files/url.txt','r') as urls:
    for url in urls.readlines():
        req = url.strip()
        page=requests.get(req)
        time.sleep(1)
        if(page.status_code == 200):
            text_file.write(url)
text_file.close()

Checking the status code of each URL. If status code = 200 then the URL works and if the status code = 404 then the URL does not work.

In [18]:
with open('../data/Txt_files/Pypi_url.txt') as f:
    size=sum(1 for _ in f)

In [19]:
size

4243

There are 4243 working Pypi URLs.

The cell below takes 30 mins to run

In [17]:
j = 0
#send every uri to the api and write the respsones to a textfile
with open('../data/Txt_files/Pypi_url.txt', 'r') as f2:
    for i in f2:
        j += 1
        uri = i.strip(' \n\t')
        r = requests.get(i)
        data = r.json()
        with open('../data/Pypi_response/response_%s.json' %j, 'w') as text_file:
            json.dump(data, text_file)
            time.sleep(1)
            text_file.close()
    # save the result to a json file

Printing contents of URL pages into separate JSON files.

In [20]:
length = 4239
data1 = []
data2 = []
import json
for j in range(length):
    j += 1
    with open('../data/Pypi_response/response_%s.json' %j) as json_data:
        d = json.load(json_data)
        data1.append(d['info']['name'])
        data2.append(d['info']['description'])
        
descrip = pd.DataFrame(data1, data2)

Getting name and descriptions of each import from their respective JSON file.

In [21]:
length = 4239
data3 = []
import json
for j in range(length):
    j += 1
    with open('../data/Pypi_response/response_%s.json' %j) as json_data:
        d = json.load(json_data)
        data3.append(d['info']['summary'])
        
summary = pd.DataFrame(data3)

Extracting summary information in case description is empty or 'UNKNOWN'

In [22]:
descrip = descrip.reset_index()
descrip.columns = ['Description', 'Name']
descrip.head()

,Description,Name
0,It provides:\n\n- a powerful N-dimensional arr...,numpy
1,"**pandas** is a Python package providing fast,...",pandas
2,Use `scikit-learn <https://pypi.python.org/pyp...,sklearn
3,Matplotlib strives to produce publication qual...,matplotlib
4,Seaborn is a library for making statistical gr...,seaborn


In [23]:
result = pd.concat([descrip, summary], axis=1)

In [24]:
result.columns = ['Description', 'Name', 'Summary']

In [25]:
result.head()

,Description,Name,Summary
0,It provides:\n\n- a powerful N-dimensional arr...,numpy,NumPy is the fundamental package for array com...
1,"**pandas** is a Python package providing fast,...",pandas,"Powerful data structures for data analysis, ti..."
2,Use `scikit-learn <https://pypi.python.org/pyp...,sklearn,A set of python modules for machine learning a...
3,Matplotlib strives to produce publication qual...,matplotlib,Python plotting package
4,Seaborn is a library for making statistical gr...,seaborn,seaborn: statistical data visualization


Anywhere description is UNKNOWN or empty I add the summary to that column

In [26]:
result.loc[result["Description"]== '','Description'] = result["Summary"]

In [27]:
result.loc[result["Description"]== 'UNKNOWN','Description'] = result["Summary"]

In [28]:
result.loc[result["Description"]== 'UNKNOWN\n\n\n','Description'] = result["Summary"]

In [29]:
result.loc[result["Description"]== 'None','Description'] = result["Summary"]

In [30]:
result.loc[result["Description"]== 'NaN','Description'] = result["Summary"]

In [36]:
result['Summary'] = result['Summary'].replace('UNKNOWN', np.nan)

In [37]:
result['Description'] = result['Description'].replace('UNKNOWN', np.nan)

### Saving to CSV file

In [38]:
result.to_csv('../data/CSV_files/pypi_descrip.csv', index=False, encoding = 'utf-8')

This file will be used for analysis in Analysis_300